In [36]:
import torch 
from transformers import RobertaTokenizer, RobertaForSequenceClassification 
from torch.nn.functional import softmax

In [37]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)




config.json: 100%|██████████| 747/747 [00:00<00:00, 747kB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 25.0MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 54.3MB/s]
special_tokens_map.json: 100%|██████████| 150/150 [00:00<?, ?B/s] 
pytorch_model.bin: 100%|██████████| 499M/499M [00:07<00:00, 66.0MB/s] 


In [46]:
def get_roberta_score(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        return l, np.round(float(s), 4)

In [50]:
import os 
import glob 
import pandas as pd
data_path = 'twitter_data/data/'
files = os.path.join(data_path, '*.csv')
for file in glob.glob(files):
    df = pd.read_csv(file)
    df['Content'] = df['Content'].apply(lambda x: x if isinstance(x, str) else '')
    sentiment_results = [get_roberta_score(tweet) for tweet in df['Content']]
    df['roberta_sentiment_label'] = [result[0] for result in sentiment_results]
    df['roberta_sentiment_score'] = [result[1] for result in sentiment_results]
    df.to_csv(file, index=False)